In [1]:
import pandas as pd

In [24]:
dee = pd.read_excel('data/deezer-data_2175171744.xlsx', sheet_name='10_listeningHistory', )
dee = dee.drop(columns=['Platform Name', 'Platform Model'])
dee.Date = pd.to_datetime(dee.Date,format='%Y-%m-%d %H:%M:%S')
dee = dee.rename(columns={"Listening Time" : "Listening Time (s)"})
dee['conn_country']=dee['ISRC'].apply(lambda x: x[0:2])


In [25]:
dee.head()

,Song Title,Artist,ISRC,Album Title,IP Address,Listening Time (s),Date,conn_country
0,"Les Rois Du Monde (Thème De ""Roméo Et Juliette...",Best Of Hits,FR0W61000893,Best Of Hits Vol. 74,37.164.130.31,198,2021-08-09 14:50:11,FR
1,We Found Love,"Rihanna, Calvin Harris",USUM71115507,Talk That Talk,93.12.51.200,1,2023-05-05 17:14:06,US
2,We Found Love,"Rihanna, Calvin Harris",USUM71115507,Talk That Talk,37.170.143.159,214,2023-05-08 11:26:27,US
3,We Found Love,"Rihanna, Calvin Harris",USUM71115507,Talk That Talk,147.161.184.198,216,2023-05-16 11:48:39,US
4,Beggin,Madcon,NOBLV0703011,Beggin,84.102.140.147,218,2020-03-17 18:16:50,NO


In [37]:
spot = pd.read_json('data/Spotify_Audio_2013-2024_251124.json')
spot.head()

,ts,platform,ms_played,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,episode_show_name,spotify_episode_uri,reason_start,reason_end,shuffle,skipped,offline,offline_timestamp,incognito_mode
0,2013-05-14T17:02:38Z,WebPlayer (websocket RFC6455),221999,FR,81.57.133.167,Always On The Run,Yuksek,Living On The Edge Of Time,spotify:track:3McVJ4pdwNMP4RJtsJAogG,None,None,None,autoplay,nextbtn,False,False,False,NaN,False
1,2013-05-14T17:04:37Z,Windows 7 (Unknown Ed) SP1 [x86 0],56188,FR,81.57.133.167,Can't Hold Us (feat. Ray Dalton),Macklemore & Ryan Lewis,The Heist,spotify:track:3BE0getU08uXas8ZjULABA,None,None,None,unknown,popup,False,True,False,NaN,False
2,2013-05-14T17:05:15Z,Windows 7 (Unknown Ed) SP1 [x86 0],39020,FR,81.57.133.167,Suit & Tie (feat. Jay-Z),Justin Timberlake,The 20/20 Experience (Deluxe Version),spotify:track:6vt0I1cw1YmAIKDJvHVIM5,None,None,None,popup,popup,False,True,False,NaN,False
3,2013-05-14T17:05:19Z,Windows 7 (Unknown Ed) SP1 [x86 0],3947,FR,81.57.133.167,Down The Road,C2C,Tetra,spotify:track:2cx9XbBUC76pi3ZpGk1MJ5,None,None,None,popup,popup,False,True,False,NaN,False
4,2013-05-14T17:05:45Z,Windows 7 (Unknown Ed) SP1 [x86 0],26225,FR,81.57.133.167,Hot Right Now (feat. RITA ORA),DJ Fresh,ORA,spotify:track:4eY1fWFPLztKDx9athN4hx,None,None,None,popup,popup,False,True,False,NaN,False


In [39]:


spot['Listening Time (s)']=round(spot['ms_played']/1000,0)
spot = spot.drop(columns=['platform', 'ms_played'])

spot = spot.rename(columns={'ts': 'Date', 
                     'ip_addr':"IP Address",
                      'master_metadata_track_name': 'Song Title',
                      'master_metadata_album_artist_name' : 'Album Artist',
                      'master_metadata_album_album_name' : 'Album Title'})

spot.Date = pd.to_datetime(spot.Date, format ="%Y-%m-%dT%H:%M:%SZ" )
spot = spot.iloc[:,0:7]

In [57]:
data = pd.concat([dee,spot])
data['Year'] = data['Date'].dt.year
data = data[data.Year > 2018]
data['spotify_track_uri'] = data['spotify_track_uri'].astype(str) 
data['Platform']=data['spotify_track_uri'].apply(lambda x: "Deezer" if x == 'nan' else "Spotify")
data = data[data['Song Title'].isna()== False]
data.head()

,Song Title,Artist,ISRC,Album Title,IP Address,Listening Time (s),Date,conn_country,Album Artist,spotify_track_uri,Year,Platform
0,"Les Rois Du Monde (Thème De ""Roméo Et Juliette...",Best Of Hits,FR0W61000893,Best Of Hits Vol. 74,37.164.130.31,198.0,2021-08-09 14:50:11,FR,NaN,nan,2021,Deezer
1,We Found Love,"Rihanna, Calvin Harris",USUM71115507,Talk That Talk,93.12.51.200,1.0,2023-05-05 17:14:06,US,NaN,nan,2023,Deezer
2,We Found Love,"Rihanna, Calvin Harris",USUM71115507,Talk That Talk,37.170.143.159,214.0,2023-05-08 11:26:27,US,NaN,nan,2023,Deezer
3,We Found Love,"Rihanna, Calvin Harris",USUM71115507,Talk That Talk,147.161.184.198,216.0,2023-05-16 11:48:39,US,NaN,nan,2023,Deezer
4,Beggin,Madcon,NOBLV0703011,Beggin,84.102.140.147,218.0,2020-03-17 18:16:50,NO,NaN,nan,2020,Deezer
